## Question Generation

In [4]:
# collect all paragraphs
path = '../data/cast19/test_paragraphs.tsv'
qp_ids = {}  # passage id to question id map
paras = {}
with open(path, encoding='utf-8') as f:
    for i, line in enumerate(f):
        q_id, para_id, para_text = line.rstrip().split('\t')
        if para_id not in qp_ids:
            qp_ids[para_id] = []
        qp_ids[para_id].append(q_id)
        if para_id not in paras:
            paras[para_id] = para_text
print(len(paras))
print(len(qp_ids))

6485
6485


In [15]:
# collect all GS questions
path_qs = '../data/cast19/test_questions.tsv'

gs_qs = {}
with open(path_qs, 'r') as f:
#     next(f)
    for line in f:
        q_id, q = line.split('\t')
#         print(q_id)
        gs_qs[q_id] = (q.strip('\n'))
print(len(gs_qs))
# print(gs_qs['11_1'])

479


In [16]:
# generate questions for each paragraph separately
import datetime
import json
import os

squash_path = '/home/svakule/squash-generation'

top_p = 0.9
gen_frac = 0.5
spec_frac = 0.8


for p_id, p_text in paras.items():
    key = p_id
    
    metadata = {
            "input_text": p_text,
            "key": key,
            "timestamp": str(datetime.datetime.now()),
            "settings": {
                "top_p": top_p,
                "gen_frac": gen_frac,
                "spec_frac": spec_frac
            }
        }

#     os.mkdir("%s/squash/temp/trec_cast19_paragraphs/%s" % (squash_path, key))

#     with open('%s/squash/temp/trec_cast19_paragraphs/%s/metadata.json' % (squash_path, key), 'w') as f:
#         f.write(json.dumps(metadata))

In [17]:
# for d in squash/temp/trec_cast19_paragraphs/*/ ; do
#     k="$(basename $d)"
#     echo "$k"
#     python squash/extract_answers.py --key trec_cast19_paragraphs/$k
#     python question-generation/interact.py --model_checkpoint question-generation/gpt2_corefs_question_generation --model_type gpt2 --key trec_cast19_paragraphs/$k --filename squash/temp/trec_cast19_paragraphs/$d/input.pkl
# done
# vim squash/temp/trec_cast19_paragraphs/generated_questions.json

# Question Evaluation

In [18]:
# test rouge scorer
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

scores = scorer.score("what is fsa health plan .", "what is health care fsa plan .")  # reference predicted
print(scores)

{'rouge1': Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091), 'rougeL': Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272)}


In [19]:
# load questions per passage
import os
import json

squash_path = '/home/svakule/squash-generation/squash/temp/trec_cast19_paragraphs'

generated_questions = {}
for p_id in os.listdir(squash_path):
#     print(p_id)
#     
    try:
        with open(squash_path+'/%s/generated_questions.json' % p_id) as f:
            paragraphs = json.load(f)['data'][0]['paragraphs']
#             print(len(paragraphs))
            for i, p in enumerate(paragraphs):
    #             print(gs_qs[question_id])  # original (reference) question
    #             print(p['context']) # original paragraph
                generated_questions[p_id] = []
                for qa in p['qas']:
                    q = qa['question']#.lower()
    #                 print(q)
                    generated_questions[p_id].append(q)
    except:
        continue
                    
#     break
print(len(generated_questions))

6475


In [27]:
# oracle selection
import numpy as np

max_scores = []
# iterate over all passages
for p_id in qp_ids:
    if p_id in generated_questions:
        gen_questions = generated_questions[p_id]
    else:
        gen_questions = []
    gt_questions = [gs_qs[q_id] for q_id in qp_ids[p_id]]
#     print(gt_questions)
#     print(gen_questions)
    for gt_question in gt_questions:
        all_q_gens_scores = [0]
        for gen_question in gen_questions:
            scores = scorer.score(gt_question, gen_question)  # reference predicted
            rouge1f = scores['rouge1'].fmeasure
            all_q_gens_scores.append(scores['rouge1'].fmeasure)
        max_score = max(all_q_gens_scores)
        # show max scoring question
#         print(gt_question)
#         q_decoded = gen_questions[np.argmax(all_q_gens_scores)]
#         print(q_decoded)
    #     print('\n')
        max_scores.append(max_score)
#     break
# print(max_scores)
print(np.mean(max_scores))

0.493592177081424


In [28]:
print(len(max_scores))

8120
